<a href="https://colab.research.google.com/github/nipuni1313/CNN-for-image-classification/blob/main/2_cnn_ass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [ ]:
# Constants
IMG_SIZE = 224  # Image size for ResNet50 and VGG16
BATCH_SIZE = 32
EPOCHS = 20
NUM_CLASSES = 6  # Update this based on your dataset
DATASET_PATH = '/content/drive/MyDrive/RealWasteDataset_D/'

In [ ]:
# Step 1: Split dataset into training, validation, and testing
train_dir = os.path.join(DATASET_PATH, 'train')
val_dir = os.path.join(DATASET_PATH, 'validation')
test_dir = os.path.join(DATASET_PATH, 'test')

# Step 2: Data Generators for Augmentation and Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
)

val_test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_data = train_datagen.flow_from_directory(
    train_dir, target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE, class_mode='categorical'
)
val_data = val_test_datagen.flow_from_directory(
    val_dir, target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE, class_mode='categorical'
)
test_data = val_test_datagen.flow_from_directory(
    test_dir, target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE, class_mode='categorical'
)

In [ ]:
# Step 3: Model Creation Function
def build_model(base_model, num_classes, dropout_rate=0.2, trainable_layers=0):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    base_model.trainable = False  # Freeze the base model initially

    # Custom Top Layers
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D(name="avg_pool")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(num_classes, activation="softmax", name="pred")(x)

    model = models.Model(inputs, outputs, name=f"{base_model.name}_FineTuned")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
    return model

# Step 4: Train and Evaluate Models
def train_and_evaluate(model, train_data, val_data, test_data, epochs, model_name):
    history = model.fit(train_data, validation_data=val_data, epochs=epochs)

    test_loss, test_acc = model.evaluate(test_data)
    print(f"{model_name} Test Accuracy: {test_acc:.2f}")

    return history, test_acc

# Step 5: Plot Training History
def plot_history(histories, test_accuracies, model_names):
    plt.figure(figsize=(14, 7))
    for history, test_acc, name in zip(histories, test_accuracies, model_names):
        plt.plot(history.history['accuracy'], label=f'{name} Train Accuracy')
        plt.plot(history.history['val_accuracy'], label=f'{name} Validation Accuracy')
        plt.axhline(y=test_acc, linestyle='--', label=f'{name} Test Accuracy')

    plt.title("Training, Validation, and Test Accuracies")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.grid(True)
    plt.show()

# Step 6: Fine-Tune Models
def fine_tune_model(model, unfreeze_layers, train_data, val_data, test_data, epochs):
    # Unfreeze the top layers for fine-tuning
    for layer in model.layers[-unfreeze_layers:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

    history = model.fit(train_data, validation_data=val_data, epochs=epochs)
    test_loss, test_acc = model.evaluate(test_data)
    print(f"Fine-Tuned Test Accuracy: {test_acc:.2f}")
    return history, test_acc


In [ ]:
# Step 7: Main Execution
base_resnet = ResNet50(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3), pooling='avg')
base_vgg = VGG16(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3), pooling='avg')

# Initialize models
resnet_model = build_model(base_resnet, NUM_CLASSES)
vgg_model = build_model(base_vgg, NUM_CLASSES)

In [ ]:
# Train resnet
history_resnet, resnet_test_acc = train_and_evaluate(resnet_model, train_data, val_data, test_data, EPOCHS, "ResNet50")

In [ ]:
#Train vgg
history_vgg, vgg_test_acc = train_and_evaluate(vgg_model, train_data, val_data, test_data, EPOCHS, "VGG16")

In [ ]:
# Fine-tune models
history_resnet_fine, resnet_test_acc_fine = fine_tune_model(resnet_model, unfreeze_layers=20, train_data=train_data, val_data=val_data, test_data=test_data, epochs=5)

In [ ]:
# Fine-tune models
history_vgg_fine, vgg_test_acc_fine = fine_tune_model(vgg_model, unfreeze_layers=20, train_data=train_data, val_data=val_data, test_data=test_data, epochs=5)


In [ ]:
# Plot histories
plot_history(
    [history_resnet, history_resnet_fine, history_vgg, history_vgg_fine],
    [resnet_test_acc_fine, vgg_test_acc_fine],
    ["ResNet50", "ResNet50 (Fine-Tuned)", "VGG16", "VGG16 (Fine-Tuned)"]
)